In [35]:
import os
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.multitest import fdrcorrection
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Tuple
import logging

In [47]:
# Global configurations
code_directory = '/Users/tereza/nishant/atlas/atlas_work_terez/atlas_harmonization/code'
os.chdir(code_directory)

# RESULTS_DIR = '../results'
# DATA_DIR = '../Data'
# FIGURES_DIR = '../figures'

# if not os.path.exists(FIGURES_DIR):
#     os.makedirs(FIGURES_DIR)

In [48]:
RESULTS_DIR = '../results'
DATA_DIR = '../Data'
FIGURES_DIR = '../figures'

In [23]:
# # Feature definitions
# FEATURE_COLUMNS = [
#     'deltaRel_mean', 
#     'thetaRel_mean', 
#     'alphaRel_mean', 
#     'betaRel_mean',
#     'gammaRel_mean',
#     'entropy_1min_mean',
#     'entropy_fullts_mean'
# ]

# FEATURE_NAMES = {
#     'deltaRel_mean': 'Delta Band (0.5-4 Hz)',
#     'thetaRel_mean': 'Theta Band (4-8 Hz)',
#     'alphaRel_mean': 'Alpha Band (8-13 Hz)',
#     'betaRel_mean': 'Beta Band (13-30 Hz)',
#     'gammaRel_mean': 'Gamma Band (30-80 Hz)',
#     'entropy_1min_mean': 'Signal Entropy (1-min)',
#     'entropy_fullts_mean': 'Signal Entropy (full)'
# }

In [51]:
class RegionalAnalysis:
    def __init__(self):
        self.logger = logging.getLogger(__name__)
        self.feature_columns = [
            'deltaRel_mean', 
            'thetaRel_mean', 
            'alphaRel_mean', 
            'betaRel_mean',
            'gammaRel_mean',
            'entropy_1min_mean',
            'entropy_fullts_mean'
        ]
        
    def prepare_data(self, region: str, feature: str) -> Tuple[np.ndarray, np.ndarray]:
        """
        Prepare data for unpaired statistical comparison
        """
        # Get data for specific region
        hup_region_data = self.hup_features[self.hup_features['roi'] == region]
        mni_region_data = self.mni_features[self.mni_features['roi'] == region]
        
        # Group by patient and get mean values
        hup_data = hup_region_data.groupby('patient_id')[feature].mean().values
        mni_data = mni_region_data.groupby('patient_id')[feature].mean().values
        
        # Check minimum sample sizes
        if len(hup_data) < 5 or len(mni_data) < 5:
            raise ValueError(f"Insufficient samples for region {region} (HUP: {len(hup_data)}, MNI: {len(mni_data)})")
            
        return hup_data, mni_data
    
    def compute_effect_size(self, hup_data: np.ndarray, mni_data: np.ndarray) -> float:
        """
        Compute Cohen's d effect size for unpaired data
        """
        n1, n2 = len(hup_data), len(mni_data)
        var1, var2 = np.var(hup_data, ddof=1), np.var(mni_data, ddof=1)
        
        # Pooled standard deviation
        pooled_se = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))
        
        # Effect size (Cohen's d)
        d = (np.mean(hup_data) - np.mean(mni_data)) / pooled_se
        return d
    
    def analyze_regions(self) -> pd.DataFrame:
        """
        Perform unpaired regional analysis between HUP and MNI cohorts
        """
        results = []
        
        for region in self.common_regions:
            self.logger.info(f"Analyzing region: {region}")
            
            for feature in self.feature_columns:
                try:
                    # Get data
                    hup_data, mni_data = self.prepare_data(region, feature)
                    
                    # Perform Mann-Whitney U test (unpaired)
                    statistic, pvalue = stats.mannwhitneyu(
                        hup_data, 
                        mni_data,
                        alternative='two-sided'
                    )
                    
                    # Compute effect size
                    effect_size = self.compute_effect_size(hup_data, mni_data)
                    
                    results.append({
                        'region': region,
                        'feature': feature,
                        'statistic': statistic,
                        'pvalue': pvalue,
                        'effect_size': effect_size,
                        'hup_mean': np.mean(hup_data),
                        'mni_mean': np.mean(mni_data),
                        'hup_std': np.std(hup_data),
                        'mni_std': np.std(mni_data),
                        'hup_n': len(hup_data),
                        'mni_n': len(mni_data)
                    })
                    
                except ValueError as e:
                    self.logger.warning(f"Skipping {region}-{feature}: {str(e)}")
                except Exception as e:
                    self.logger.error(f"Error processing {region}-{feature}: {str(e)}")
        
        # Create results DataFrame
        results_df = pd.DataFrame(results)
        
        if len(results_df) > 0:
            # Apply FDR correction across all tests
            _, fdr_pvals = fdrcorrection(results_df['pvalue'])
            results_df['pvalue_fdr'] = fdr_pvals
        
        return results_df

    def summarize_results(self, results_df: pd.DataFrame):
        """
        Print summary of findings
        """
        if len(results_df) == 0:
            print("No results to summarize")
            return
            
        print("\nRegional Analysis Summary")
        print("=" * 50)
        print(f"\nTotal tests performed: {len(results_df)}")
        print(f"Number of regions tested: {results_df['region'].nunique()}")
        print(f"Number of features tested: {results_df['feature'].nunique()}")
        
        sig_results = results_df[results_df['pvalue_fdr'] < 0.05]
        print(f"\nSignificant results after FDR correction: {len(sig_results)}")
        
        if len(sig_results) > 0:
            print("\nSignificant differences found:")
            for feature in self.feature_columns:
                feature_results = sig_results[sig_results['feature'] == feature]
                if len(feature_results) > 0:
                    print(f"\n{feature}:")
                    for _, row in feature_results.iterrows():
                        direction = "higher in HUP" if row['effect_size'] > 0 else "higher in MNI"
                        print(f"  * {row['region']}: {direction}")
                        print(f"    Effect size (d): {row['effect_size']:.3f}")
                        print(f"    p-value (FDR): {row['pvalue_fdr']:.3e}")
                        print(f"    Samples: HUP={row['hup_n']}, MNI={row['mni_n']}")

In [49]:
# class RegionalAnalysis:
#     def __init__(self):
#         self.logger = logging.getLogger(__name__)
#         self.feature_columns = [
#             'deltaRel_mean', 
#             'thetaRel_mean', 
#             'alphaRel_mean', 
#             'betaRel_mean',
#             'gammaRel_mean',
#             'entropy_1min_mean',
#             'entropy_fullts_mean'
#         ]
        
#         self.feature_names = {
#             'deltaRel_mean': 'Delta Band (0.5-4 Hz)',
#             'thetaRel_mean': 'Theta Band (4-8 Hz)',
#             'alphaRel_mean': 'Alpha Band (8-13 Hz)',
#             'betaRel_mean': 'Beta Band (13-30 Hz)',
#             'gammaRel_mean': 'Gamma Band (30-80 Hz)',
#             'entropy_1min_mean': 'Signal Entropy (1-min)',
#             'entropy_fullts_mean': 'Signal Entropy (full)'
#         }
        
#     def load_data(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
#         """
#         Load and prepare the regional feature data for both cohorts
#         """
#         # Load data
#         self.hup_features = pd.read_csv("ge_go_hup_region_features.csv")
#         self.mni_features = pd.read_csv("mni_region_features.csv")
        
#         # Get common regions
#         self.common_regions = set(self.hup_features['roi'].unique()) & set(self.mni_features['roi'].unique())
        
#         print(f"Loaded data:")
#         print(f"HUP features shape: {self.hup_features.shape}")
#         print(f"MNI features shape: {self.mni_features.shape}")
#         print(f"Number of common regions: {len(self.common_regions)}")
        
#         return self.hup_features, self.mni_features
    
#     def prepare_paired_data(self, region: str, feature: str) -> Tuple[np.ndarray, np.ndarray]:
#         """
#         Prepare paired data for a specific region and feature.
#         Returns arrays of equal length for valid statistical comparison.
#         """
#         # Get data for specific region
#         hup_data = self.hup_features[self.hup_features['roi'] == region][feature].values
#         mni_data = self.mni_features[self.mni_features['roi'] == region][feature].values
        
#         # Get minimum length to ensure paired data
#         min_length = min(len(hup_data), len(mni_data))
        
#         if min_length < 5:  # Minimum sample size requirement
#             raise ValueError(f"Insufficient samples for region {region}")
            
#         # Match lengths for pairing
#         hup_data = hup_data[:min_length]
#         mni_data = mni_data[:min_length]
        
#         return hup_data, mni_data
    
#     def compute_effect_size(self, hup_data: np.ndarray, mni_data: np.ndarray) -> float:
#         """
#         Compute Cohen's d effect size for paired data
#         """
#         diff = hup_data - mni_data
#         d = np.mean(diff) / np.std(diff)
#         return d
    
#     def analyze_regions(self) -> pd.DataFrame:
#         """
#         Perform paired regional analysis between HUP and MNI cohorts
#         """
#         results = []
        
#         for region in self.common_regions:
#             self.logger.info(f"Analyzing region: {region}")
            
#             for feature in self.feature_columns:
#                 try:
#                     # Get paired data
#                     hup_data, mni_data = self.prepare_paired_data(region, feature)
                    
#                     # Perform Wilcoxon signed-rank test (paired test)
#                     statistic, pvalue = stats.wilcoxon(hup_data, mni_data)
                    
#                     # Compute effect size
#                     effect_size = self.compute_effect_size(hup_data, mni_data)
                    
#                     results.append({
#                         'region': region,
#                         'feature': feature,
#                         'statistic': statistic,
#                         'pvalue': pvalue,
#                         'effect_size': effect_size,
#                         'hup_mean': np.mean(hup_data),
#                         'mni_mean': np.mean(mni_data),
#                         'hup_std': np.std(hup_data),
#                         'mni_std': np.std(mni_data),
#                         'n_samples': len(hup_data)
#                     })
                    
#                 except ValueError as e:
#                     self.logger.warning(f"Skipping {region}-{feature}: {str(e)}")
#                 except Exception as e:
#                     self.logger.error(f"Error processing {region}-{feature}: {str(e)}")
        
#         # Create results DataFrame
#         results_df = pd.DataFrame(results)
        
#         # Apply FDR correction
#         _, fdr_pvals = fdrcorrection(results_df['pvalue'])
#         results_df['pvalue_fdr'] = fdr_pvals
        
#         return results_df
    
#     def summarize_results(self, results_df: pd.DataFrame):
#         """
#         Print summary of significant findings
#         """
#         print("\nRegional Analysis Summary")
#         print("=" * 50)
        
#         for feature in self.feature_columns:
#             feature_results = results_df[results_df['feature'] == feature]
#             sig_results = feature_results[feature_results['pvalue_fdr'] < 0.05]
            
#             print(f"\n{self.feature_names[feature]}:")
#             print(f"- {len(sig_results)} regions show significant differences")
            
#             if len(sig_results) > 0:
#                 sig_results = sig_results.sort_values('effect_size', key=abs, ascending=False)
#                 print("\nTop regions with largest differences:")
                
#                 for _, row in sig_results.head(3).iterrows():
#                     direction = "higher in HUP" if row['effect_size'] > 0 else "higher in MNI"
#                     effect_mag = "large" if abs(row['effect_size']) > 0.8 else \
#                                 "medium" if abs(row['effect_size']) > 0.5 else "small"
                    
#                     print(f"  * {row['region']}: {effect_mag} effect {direction}")
#                     print(f"    (p={row['pvalue_fdr']:.3e}, d={row['effect_size']:.2f}, n={row['n_samples']})")


In [52]:
def main():
    # Set up logging
    logging.basicConfig(level=logging.INFO)
    
    try:
        # Initialize analysis
        analysis = RegionalAnalysis()
        
        # Load data
        analysis.load_data()
        
        # Perform analysis
        results = analysis.analyze_regions()
        
        # Summarize results
        analysis.summarize_results(results)
        
        # Save results
        output_path = os.path.join(RESULTS_DIR, 'regional_analysis_results.csv')
        # results.to_csv(output_path, index=False)
        print(f"\nResults saved to: {output_path}")
        
    except Exception as e:
        print(f"Error in main execution: {str(e)}")
        raise

if __name__ == "__main__":
    main()

Error in main execution: 'RegionalAnalysis' object has no attribute 'load_data'


AttributeError: 'RegionalAnalysis' object has no attribute 'load_data'

In [44]:
def analyze_regional_coverage():
    """
    Analyze regional coverage and patient counts in both cohorts
    """
    # Load data
    hup_features = pd.read_csv("../results/ge_go_hup_region_features.csv")
    mni_features = pd.read_csv("../results/mni_region_features.csv")
    
    print("\nHUP Dataset Analysis:")
    print("=" * 50)
    
    # Get patient and region counts for HUP
    hup_regions = hup_features.groupby('roi').agg({
        'patient_id': ['nunique', 'count']
    }).reset_index()
    hup_regions.columns = ['roi', 'n_patients', 'n_recordings']
    hup_regions = hup_regions.sort_values('n_patients', ascending=False)
    
    print(f"\nTotal unique regions in HUP: {len(hup_regions)}")
    print(f"Total unique patients in HUP: {hup_features['patient_id'].nunique()}")
    print("\nPatient counts per region (top 10):")
    print(hup_regions.head(10))
    
    print("\nRegions with < 5 patients:")
    print(hup_regions[hup_regions['n_patients'] < 5][['roi', 'n_patients']])
    
    print("\nMNI Dataset Analysis:")
    print("=" * 50)
    
    # Get patient and region counts for MNI
    mni_regions = mni_features.groupby('roi').agg({
        'patient_id': ['nunique', 'count']
    }).reset_index()
    mni_regions.columns = ['roi', 'n_patients', 'n_recordings']
    mni_regions = mni_regions.sort_values('n_patients', ascending=False)
    
    print(f"\nTotal unique regions in MNI: {len(mni_regions)}")
    print(f"Total unique patients in MNI: {mni_features['patient_id'].nunique()}")
    print("\nPatient counts per region (top 10):")
    print(mni_regions.head(10))
    
    print("\nRegions with < 5 patients:")
    print(mni_regions[mni_regions['n_patients'] < 5][['roi', 'n_patients']])
    
    # Analyze overlap
    common_regions = set(hup_regions['roi']) & set(mni_regions['roi'])
    print(f"\nOverlap Analysis:")
    print("=" * 50)
    print(f"Number of common regions: {len(common_regions)}")
    
    # Analyze which regions have sufficient patients in both cohorts
    valid_regions = []
    for region in common_regions:
        hup_patients = hup_regions[hup_regions['roi'] == region]['n_patients'].iloc[0]
        mni_patients = mni_regions[mni_regions['roi'] == region]['n_patients'].iloc[0]
        
        if hup_patients >= 5 and mni_patients >= 5:
            valid_regions.append({
                'region': region,
                'hup_patients': hup_patients,
                'mni_patients': mni_patients
            })
    
    print(f"\nRegions with ≥5 patients in both cohorts: {len(valid_regions)}")
    print("\nValid regions for paired analysis:")
    valid_df = pd.DataFrame(valid_regions)
    print(valid_df.sort_values('hup_patients', ascending=False))

if __name__ == "__main__":
    analyze_regional_coverage()


HUP Dataset Analysis:

Total unique regions in HUP: 67
Total unique patients in HUP: 28

Patient counts per region (top 10):
                            roi  n_patients  n_recordings
61  ctx-rh-rostralmiddlefrontal          16            16
23        ctx-lh-middletemporal          15            15
17      ctx-lh-inferiortemporal          15            15
38      ctx-lh-superiortemporal          14            14
44              ctx-rh-fusiform          14            14
46      ctx-rh-inferiortemporal          13            13
52        ctx-rh-middletemporal          13            13
15              ctx-lh-fusiform          13            13
35  ctx-lh-rostralmiddlefrontal          13            13
58            ctx-rh-precentral          12            12

Regions with < 5 patients:
                                roi  n_patients
18                    ctx-lh-insula           4
36           ctx-lh-superiorfrontal           4
0                     Left-Amygdala           4
47              

In [41]:
def print_analysis_methodology():
    """
    Print detailed summary of statistical methodology and findings in the console
    """
    print("\nStatistical Analysis Methodology:")
    print("=" * 50)
    print("\n1. Sample Size Analysis:")
    print("   - HUP Cohort: 28 unique patients across 67 regions")
    print("   - MNI Cohort: 106 unique patients across 68 regions")
    print("   - 57 overlapping regions between cohorts")
    print("   - 33 regions had sufficient data (≥5 patients in both cohorts)")
    
    print("\n2. Statistical Testing Pipeline:")
    print("   Current Implementation:")
    print("   a) Data preparation:")
    print("      - Group by patient within each region")
    print("      - Match patient counts between cohorts")
    print("      - Minimum requirement: 5 patients per group")
    
    print("\n   b) For each region and feature:")
    print("      - Wilcoxon signed-rank test (paired, two-tailed)")
    print("      - Effect size computation (Cohen's d for paired data)")
    print("      - FDR correction for multiple comparisons")
    
    print("\n3. Potential Improvements Needed:")
    print("   a) Patient Matching:")
    print("      - Current implementation truncates to equal lengths")
    print("      - Should use consistent patient subset across features")
    
    print("   b) Multiple Comparison Correction:")
    print("      - Currently using FDR across all tests")
    print("      - Consider hierarchical testing strategy:")
    print("        * First test global differences")
    print("        * Then test individual regions if global is significant")
    
    print("   c) Effect Size Interpretation:")
    print("      - Add confidence intervals for effect sizes")
    
    print("\n4. Interpretation of Current Results:")
    print("   - No regions showed significant differences after correction")
    print("   - Possible explanations:")
    print("     a) True absence of site-specific differences")
    print("     b) Insufficient power due to small sample sizes")
    print("     c) High variability within regions")
    print("     d) Over-conservative multiple comparison correction")

if __name__ == "__main__":
    print_analysis_methodology()


Statistical Analysis Methodology:

1. Sample Size Analysis:
   - HUP Cohort: 28 unique patients across 67 regions
   - MNI Cohort: 106 unique patients across 68 regions
   - 57 overlapping regions between cohorts
   - 33 regions had sufficient data (≥5 patients in both cohorts)

2. Statistical Testing Pipeline:
   Current Implementation:
   a) Data preparation:
      - Group by patient within each region
      - Match patient counts between cohorts
      - Minimum requirement: 5 patients per group

   b) For each region and feature:
      - Wilcoxon signed-rank test (paired, two-tailed)
      - Effect size computation (Cohen's d for paired data)
      - FDR correction for multiple comparisons

3. Potential Improvements Needed:
   a) Patient Matching:
      - Current implementation truncates to equal lengths
      - Should use consistent patient subset across features
   b) Multiple Comparison Correction:
      - Currently using FDR across all tests
      - Consider hierarchical testin